# Benchmarking LSTMs

This notebook contains experiments on 3 different sequence tasks:
* __Sequence Labelling__
    * A many-to-one, _Sentiment Analysis_ task on the IMDb dataset available with _torchtext_.
* __Sequence to Sequence - Same__
    * A many-to-many-same, _Predict missing word_ task on the Facebook bAbi dataset.
* __Sequence to Sequence - Different__
    * A many-to-one-different, _NTM toy_ task.
    
For each of the tasks, we will run our implementation of vanilla LSTM. <br>

### Loading dependencies

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import IMDB
from torchtext import data
from torchtext.vocab import GloVe

import random
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, classification_report

from IPython.display import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 42

# Task 1: Sequence Labelling

All code pertaining to this task will have a pre-/post-fix **SeqLabel**.

### Parameters

#### Architecture

In [1]:
# Embedding dimension/dimension for LSTM cell inputs
embed_dim = 300
# Number of hidden nodes
hidden_dim = 256
# Number of output nodes
output_dim = 1
# Number of LSTMs cells to be stacked
layers = 1
# Boolean value for bidirectioanl or not
bidirectional = False

#### Training

In [3]:
batch_size = 8
# Percentage of training data
split_ratio = 0.8
learning_rate = 0.001
epochs = 1 #200

### Loading and preparing IMDb data

In [ ]:
from imdb import IMDB_dataset

imdb = IMDB_dataset(split_ratio, seed)
imdb.load(verbose = True)
imdb.build_vocab(embed_dim)
train_loader, valid_loader, test_loader = imdb.create_data_loader(batch_size, 
                                                                  device)
vocab_len = len(imdb.TEXT.vocab)

Training data size:    20000
Validation data size:  5000
Test data size:        25000


### Training Model

In [ ]:
# Our implementation

from seq_label import LSTM_SeqLabel, SeqLabel

# Initializing model
model = LSTM_SeqLabel(vocab_len, embed_dim, hidden_dim, output_dim, 
                       imdb.pretrained_weights, layers, bidirectional)
model.to(device)

# Initializing optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_criterion = nn.BCEWithLogitsLoss()

# Initializing task
task = SeqLabel(model, optimizer, loss_criterion, device)

# Training
freq = 5    # epoch interval to calculate F1 score and save models
out_dir = "results/seq_label/"
# out_dir = "/content/drive/My Drive/colab/seq_label/"
model, stats = task.train(epochs, train_loader, valid_loader, freq, out_dir)

print("=" * 50)

### Testing Model

In [ ]:
# Testing
f1_test = task.evaluate_sentiment(test_loader, verbose=True)

### Train and Test - PyTorch

In [ ]:
# PyTorch implementation

from seq_label import SentimentNetworkBaseline

# Initializing model
model = SentimentNetworkBaseline(vocab_len, embed_dim, hidden_dim, output_dim, 
                       imdb.pretrained_weights, layers, bidirectional)
model.to(device)

# Initializing optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_criterion = nn.BCEWithLogitsLoss()

# Initializing task
task = SeqLabel(model, optimizer, loss_criterion, device)

# Training
freq = 5    # epoch interval to calculate F1 score and save models
out_dir = "results/seq_label/pytorch/"
# out_dir = "/content/drive/My Drive/colab/seq_label/"
model, stats = task.train(epochs, train_loader, valid_loader, freq, out_dir)

print("=" * 50)